# Démonstration : Fonctions de Hachage et Paradoxe des Anniversaires

**Objectifs** :
- Comprendre les fonctions de hachage cryptographiques
- Démontrer le paradoxe des anniversaires
- Montrer la résistance aux collisions de SHA-256
- Attaquer MD5 (cassé)

In [ ]:
import hashlib
import secrets
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np
import time

## 1. Fonctions de Hachage Cryptographiques

**Définition** : $H : \{0,1\}^* \to \{0,1\}^n$

**Propriétés requises** :
1. **Résistance aux préimages** : Difficile de trouver $m$ tel que $H(m) = h$
2. **Résistance aux secondes préimages** : Difficile de trouver $m' \neq m$ tel que $H(m') = H(m)$
3. **Résistance aux collisions** : Difficile de trouver $m_1 \neq m_2$ tel que $H(m_1) = H(m_2)$

In [ ]:
# Fonctions de hachage disponibles
print("Fonctions de hachage disponibles dans hashlib :")
print(hashlib.algorithms_available)

print("\n" + "=" * 70)
print("COMPARAISON DES FONCTIONS DE HACHAGE")
print("=" * 70)

message = b"Hello, Cryptography!"

hash_functions = [
    ('MD5', hashlib.md5, '❌ CASSÉ (ne pas utiliser)'),
    ('SHA-1', hashlib.sha1, '❌ CASSÉ (2017, SHAttered)'),
    ('SHA-256', hashlib.sha256, '✅ Sécurisé'),
    ('SHA-3-256', hashlib.sha3_256, '✅ Sécurisé (2015, Keccak)'),
    ('BLAKE2b', hashlib.blake2b, '✅ Sécurisé (rapide)')
]

print(f"\nMessage : {message}\n")

for name, func, status in hash_functions:
    h = func(message).hexdigest()
    size = len(func().digest()) * 8
    print(f"{name:12} ({size:>3} bits) : {h[:32]}...")
    print(f"{'':12} Status    : {status}\n")

## 2. Paradoxe des Anniversaires

**Question** : Combien de personnes faut-il pour que deux aient la même date d'anniversaire avec probabilité > 50% ?

**Réponse intuitive (fausse)** : 365/2 ≈ 183 personnes

**Réponse correcte** : **23 personnes** ! (contre-intuitif)

**Calcul** : $P(\text{collision}) = 1 - P(\text{tous différents})$

$$P(\text{tous différents}) = \frac{365}{365} \cdot \frac{364}{365} \cdot \frac{363}{365} \cdots \frac{365-n+1}{365}$$

**Approximation** : $P(\text{collision}) \approx 1 - e^{-n^2/(2 \cdot 365)}$

In [ ]:
def birthday_probability(n_people, n_days=365):
    """
    Calcule la probabilité qu'au moins deux personnes aient le même anniversaire.
    
    Args:
        n_people: Nombre de personnes
        n_days: Nombre de jours dans l'année (365)
    
    Returns:
        Probabilité de collision
    """
    if n_people > n_days:
        return 1.0  # Principe des tiroirs
    
    # P(tous différents)
    p_all_different = 1.0
    for i in range(n_people):
        p_all_different *= (n_days - i) / n_days
    
    # P(au moins une collision) = 1 - P(tous différents)
    return 1 - p_all_different

# Visualisation
people = list(range(1, 71))
probabilities = [birthday_probability(n) for n in people]

plt.figure(figsize=(12, 6))
plt.plot(people, probabilities, 'b-', linewidth=2)
plt.axhline(y=0.5, color='r', linestyle='--', label='50% de probabilité')
plt.axvline(x=23, color='g', linestyle='--', label='23 personnes')
plt.xlabel('Nombre de personnes')
plt.ylabel('Probabilité de collision')
plt.title('Paradoxe des Anniversaires')
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

# Valeurs clés
print("📊 Probabilités de collision :")
for n in [10, 20, 23, 30, 40, 50, 70]:
    p = birthday_probability(n)
    print(f"   {n:2} personnes : {p:.1%}")

print(f"\n💡 Avec seulement 23 personnes : {birthday_probability(23):.1%} de probabilité !")

## 3. Application aux Fonctions de Hachage

**Généralisation** : Pour une fonction de hachage $H : \{0,1\}^* \to \{0,1\}^n$

**Complexité pour trouver une collision** : $\approx \sqrt{2^n} = 2^{n/2}$ essais (birthday attack)

**Conséquence** :
- MD5 (128 bits) : $2^{64}$ essais → **cassé en pratique**
- SHA-1 (160 bits) : $2^{80}$ essais → **cassé en 2017**
- SHA-256 (256 bits) : $2^{128}$ essais → **sécurisé** (infaisable)
- SHA-512 (512 bits) : $2^{256}$ essais → **très sécurisé**

In [ ]:
# Simulation : Birthday attack sur fonction de hachage "jouet"
def birthday_attack_simulation(output_bits=16):
    """
    Simule un birthday attack sur une fonction de hachage avec sortie réduite.
    
    Args:
        output_bits: Taille de sortie en bits (réduite pour simulation)
    
    Returns:
        Nombre d'essais avant collision
    """
    hashes = {}
    attempts = 0
    
    while True:
        # Générer message aléatoire
        message = secrets.token_bytes(16)
        
        # Calculer hash (tronqué à output_bits)
        h = hashlib.sha256(message).digest()
        h_truncated = int.from_bytes(h, 'big') % (2 ** output_bits)
        
        attempts += 1
        
        # Vérifier collision
        if h_truncated in hashes:
            return attempts, hashes[h_truncated], message
        
        hashes[h_truncated] = message

# Test avec différentes tailles
print("=" * 70)
print("SIMULATION : Birthday Attack")
print("=" * 70)
print("\nTrouver une collision (deux messages avec même hash)\n")

for bits in [12, 16, 20]:
    attempts, m1, m2 = birthday_attack_simulation(bits)
    expected = int(np.sqrt(2 ** bits) * 1.25)  # Approximation avec facteur
    
    print(f"Sortie : {bits} bits (2^{bits} = {2**bits} valeurs possibles)")
    print(f"   Théorie : ~{expected} essais (≈ √(2^{bits}))")
    print(f"   Pratique: {attempts} essais")
    print(f"   Collision trouvée ! ✅")
    print(f"   m1 = {m1.hex()[:32]}...")
    print(f"   m2 = {m2.hex()[:32]}...\n")

print("💡 La complexité est bien en O(√n), pas en O(n) !")

## 4. MD5 : Fonction Cassée

**Historique** :
- 1991 : MD5 conçu par Ron Rivest
- 1996 : Premières faiblesses théoriques
- 2004 : Première collision trouvée
- 2008 : Attaque pratique (quelques secondes)
- **Aujourd'hui** : Complètement cassé, ne JAMAIS utiliser

**Attaque** : Collision MD5 en quelques secondes (vs $2^{64}$ théorique)

In [ ]:
# Collision MD5 connue (préfixe identique)
# Ces deux messages ont le MÊME hash MD5 !

m1 = bytes.fromhex(
    "d131dd02c5e6eec4693d9a0698aff95c" +
    "2fcab58712467eab4004583eb8fb7f89" +
    "55ad340609f4b30283e488832571415a" +
    "085125e8f7cdc99fd91dbdf280373c5b" +
    "d8823e3156348f5bae6dacd436c919c6" +
    "dd53e2b487da03fd02396306d248cda0" +
    "e99f33420f577ee8ce54b67080a80d1e" +
    "c69821bcb6a8839396f9652b6ff72a70"
)

m2 = bytes.fromhex(
    "d131dd02c5e6eec4693d9a0698aff95c" +
    "2fcab50712467eab4004583eb8fb7f89" +
    "55ad340609f4b30283e4888325f1415a" +
    "085125e8f7cdc99fd91dbd7280373c5b" +
    "d8823e3156348f5bae6dacd436c919c6" +
    "dd53e23487da03fd02396306d248cda0" +
    "e99f33420f577ee8ce54b67080280d1e" +
    "c69821bcb6a8839396f965ab6ff72a70"
)

print("=" * 70)
print("COLLISION MD5 (exemple réel)")
print("=" * 70)

hash_m1 = hashlib.md5(m1).hexdigest()
hash_m2 = hashlib.md5(m2).hexdigest()

print(f"\nMessage 1 : {m1.hex()[:64]}...")
print(f"MD5(m1)   : {hash_m1}")

print(f"\nMessage 2 : {m2.hex()[:64]}...")
print(f"MD5(m2)   : {hash_m2}")

print(f"\nMessages identiques ? {m1 == m2} ❌")
print(f"Hashes identiques ?   {hash_m1 == hash_m2} ✅")

print(f"\n💥 COLLISION ! MD5 est CASSÉ")
print(f"\n⚠️  Ne JAMAIS utiliser MD5 pour la sécurité :")
print(f"   - Pas pour signatures")
print(f"   - Pas pour intégrité (HMAC-MD5 aussi faible)")
print(f"   - Pas pour stockage de mots de passe")
print(f"\n✅ Utiliser SHA-256, SHA-3, ou BLAKE2 à la place")

## 5. SHA-256 : Fonction Sécurisée

In [ ]:
# Propriétés de SHA-256
print("=" * 70)
print("SHA-256 : Fonction de Hachage Sécurisée")
print("=" * 70)

message = b"Secure message"
h = hashlib.sha256(message)

print(f"\nMessage    : {message}")
print(f"SHA-256    : {h.hexdigest()}")
print(f"Taille     : {h.digest_size} bytes = {h.digest_size * 8} bits")
print(f"Nom        : {h.name}")
print(f"Block size : {h.block_size} bytes")

# Effet avalanche : changer 1 bit change ~50% du hash
print(f"\n📊 Effet Avalanche (changer 1 bit change tout) :")

m1 = b"Hello"
m2 = b"hello"  # Un seul bit différent (H vs h)

h1 = hashlib.sha256(m1).hexdigest()
h2 = hashlib.sha256(m2).hexdigest()

print(f"   m1 = '{m1.decode()}' → {h1}")
print(f"   m2 = '{m2.decode()}' → {h2}")

# Compter bits différents
h1_bits = bin(int(h1, 16))[2:].zfill(256)
h2_bits = bin(int(h2, 16))[2:].zfill(256)
diff_bits = sum(b1 != b2 for b1, b2 in zip(h1_bits, h2_bits))

print(f"\n   Bits différents : {diff_bits}/256 ({diff_bits/256:.1%})")
print(f"   ✅ ~50% attendu pour un bon hash (effet avalanche)")

## 6. Performance

In [ ]:
def benchmark_hash_functions():
    """
    Compare les performances de différentes fonctions de hachage.
    """
    data_sizes = [1024, 10240, 102400, 1024000]  # 1KB, 10KB, 100KB, 1MB
    iterations = 1000
    
    functions = [
        ('MD5', hashlib.md5),
        ('SHA-1', hashlib.sha1),
        ('SHA-256', hashlib.sha256),
        ('SHA-3-256', hashlib.sha3_256),
        ('BLAKE2b', hashlib.blake2b)
    ]
    
    print("=" * 70)
    print("BENCHMARK : Fonctions de Hachage")
    print("=" * 70)
    print(f"Itérations : {iterations}\n")
    
    for size in data_sizes:
        data = secrets.token_bytes(size)
        print(f"Taille : {size:>7} bytes ({size//1024:>4} KB)")
        
        for name, func in functions:
            start = time.perf_counter()
            for _ in range(iterations):
                func(data).digest()
            elapsed = time.perf_counter() - start
            
            throughput = (size * iterations) / elapsed / 1_000_000
            print(f"  {name:12} : {elapsed:>6.3f}s ({throughput:>6.1f} MB/s)")
        print()

benchmark_hash_functions()

## Conclusion

**Points clés** :
- **Paradoxe des anniversaires** : Collision en $O(\sqrt{n})$, pas $O(n)$
- **Conséquence cryptographique** : Nécessité de sorties longues (≥256 bits)
- **MD5 et SHA-1** : Complètement cassés, ne JAMAIS utiliser
- **SHA-256, SHA-3, BLAKE2** : Sécurisés et recommandés
- **Effet avalanche** : Changer 1 bit change ~50% du hash

**Recommandations** :
- ✅ SHA-256 : Standard universel
- ✅ SHA-3 : Alternative moderne (Keccak)
- ✅ BLAKE2 : Très rapide, sécurisé
- ❌ MD5, SHA-1 : Obsolètes et dangereux

**Applications** :
- Intégrité de fichiers (checksums)
- Blockchain (Bitcoin utilise SHA-256)
- Stockage mots de passe (avec salage : bcrypt, Argon2)
- HMAC (authentification de messages)